In [ ]:
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
from llama_index import Document
from llama_index import VectorStoreIndex
from llama_index import LLMPredictor, PromptHelper, ServiceContext

import pandas as pd

In [ ]:
llm = LlamaCPP(
    model_path="../../../llama/llama-2-7b-chat/ggml-model-q4_0.bin",
    temperature=0.1,
    max_new_tokens=256,
    # llama2 has a context window of 4096 tokens, but we set it lower to allow for some wiggle room
    context_window=3900,
    # kwargs to pass to __call__()
    generate_kwargs={},
    # kwargs to pass to __init__()
    # set to at least 1 to use GPU
    model_kwargs={"n_gpu_layers": 1},
    # transform inputs into Llama2 format
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=True,
)

In [ ]:
def load_documents():
    wiki_scraped=pd.read_csv("../../data/turing_internal/wiki-scraped.csv")
    wiki_scraped.dropna(subset="body", inplace=True)
    wiki_scraped_text=[str(i) for i in wiki_scraped["body"].values]

    handbook_scraped=pd.read_csv("../../data/public/handbook-scraped.csv")
    handbook_scraped.dropna(subset="body", inplace=True)
    handbook_scraped_text=[str(i) for i in handbook_scraped["body"].values]

    turingacuk=pd.read_csv("../../data/public/turingacuk-no-boilerplate.csv")
    turingacuk.dropna(subset="body", inplace=True)
    turingacuk_text=[str(i) for i in turingacuk["body"].values]

    documents = [Document(text=i) for i in wiki_scraped_text]
    documents.extend([Document(text=i) for i in handbook_scraped_text])
    documents.extend([Document(text=i) for i in turingacuk_text])

    return documents

In [ ]:
documents = load_documents()

In [ ]:
len(documents)

In [ ]:
test_docs=documents[10:20]
test_docs

In [ ]:
from llama_index.node_parser import SimpleNodeParser
from llama_index.node_parser.extractors import (
    MetadataExtractor,
    # SummaryExtractor,
    # QuestionsAnsweredExtractor,
    # TitleExtractor,
    KeywordExtractor,
    # EntityExtractor,
    MetadataFeatureExtractor,
)
from llama_index.text_splitter import TokenTextSplitter

text_splitter = TokenTextSplitter(separator=" ", chunk_size=512, chunk_overlap=128)

class CustomExtractor(MetadataFeatureExtractor):
    def extract(self, nodes):
        metadata_list = [
            {
                "custom": node.metadata["document_title"]
                + "\n"
                + node.metadata["excerpt_keywords"]
            }
            for node in nodes
        ]
        return metadata_list
    
metadata_extractor = MetadataExtractor(
    extractors=[
        # TitleExtractor(nodes=5, llm=llm),
        # QuestionsAnsweredExtractor(questions=3, llm=llm),
        # EntityExtractor(prediction_threshold=0.5),
        # SummaryExtractor(summaries=["prev", "self"], llm=llm),
        KeywordExtractor(keywords=3, llm=llm),
        # CustomExtractor()
    ],
)

node_parser = SimpleNodeParser.from_defaults(
    text_splitter=text_splitter,
    metadata_extractor=metadata_extractor,
)

nodes = node_parser.get_nodes_from_documents(test_docs)

In [ ]:
def create_service_context(
        model, 
        max_input_size=1024,
        num_output=128,
        chunk_size_lim=512,
        overlap_ratio=0.1
    ):
    llm_predictor=LLMPredictor(llm=model)
    prompt_helper=PromptHelper(max_input_size,num_output,overlap_ratio,chunk_size_lim)
    service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper, embed_model="local")
    return service_context

In [ ]:
service_context = create_service_context(llm)
index = VectorStoreIndex(nodes, service_context=service_context)

In [ ]:
query_engine = index.as_query_engine()

In [ ]:
from llama_index.evaluation import ResponseEvaluator, QueryResponseEvaluator

source_evaluator = ResponseEvaluator(service_context=service_context)
query_evaluator = QueryResponseEvaluator(service_context=service_context)

In [ ]:
query="Who is Ryan Chan?"
response = query_engine.query(query)
print(response.response)

In [ ]:
print(source_evaluator.evaluate(response))
print(query_evaluator.evaluate(query, response))

In [ ]:
len(nodes)

In [ ]:
from llama_index.evaluation import DatasetGenerator

data_generator = DatasetGenerator(nodes, service_context=service_context, num_questions_per_chunk=3)
eval_questions = data_generator.generate_questions_from_nodes()


In [ ]:
eval_questions.